# DATA622 Final Project
Daniel Dittenhafer

This notebook proceeds with using the trained convolutional neural network to predict emotion for each of the face images in the provided test set.

### Import References

In [38]:
import csv
import random
import numpy as np
import emotion_model
import dwdii_transforms

random.seed(20275)

### Definitions

The model was trained on 150x150 images, therefore we will use this size of the test data as well.

In [39]:
imgDimension = []
imgDimension.append(150)
imgDimension.append(150)

### Load Kaggle Test Data

In [40]:
testDataPath = "../../facial_expressions/test"

In [47]:
import os
ls  = os.listdir(testDataPath)
len(ls)

263

In [48]:
testData, fileList = dwdii_transforms.load_test_data(testDataPath, imgResize=imgDimension, verboseFreq=20)

0.000000: y38.jpg
0.076046: y06.jpg
0.152091: BhanuPriya_60.jpg
0.228137: NM.HA2.96.jpg
0.304183: BomanIrani_9.jpg
0.380228: 98a.jpg
0.456274: KL.SA2.162.jpg
0.532319: KL.DI2.171.jpg
0.608365: 140a.jpg
0.684411: Dileep_60.jpg
0.760456: YM.AN2.62.jpg
0.836502: KM.SU2.15.jpg
0.912548: y28.jpg
0.988593: y09.jpg


In [49]:
testData.shape

(263, 150, 150)

In [50]:
testX = testData.reshape(testData.shape[0], 1, testData.shape[1],testData.shape[2])

### Define and Load Trained Model

In [51]:
# Map the emotions to integers for categorization later.
emotions = dwdii_transforms.emotionNumerics()
print emotions
print len(emotions)

model = emotion_model.emotion_model_jh_v5(len(emotions), verbose=True, 
                                        input_shape=(1,imgDimension[0],imgDimension[1]))

{'sadness': 6, 'neutral': 2, 'contempt': 7, 'disgust': 1, 'anger': 0, 'surprise': 4, 'fear': 5, 'happiness': 3}
8
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_9 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
activation_13 (Activation)         (None, 32, 143, 143)0           convolution2d_9[0][0]            
____________________________________________________________________________________________________
maxpooling2d_9 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_13[0][0]              
____________________________________________________________________________________________________
convolution2d_10 (Convolution2D)   (None, 32, 67, 67)  25632       maxpooling2

In [52]:
loadWeights = True
if loadWeights:
    #model.load_weights("dwdii-emo-150-jhv5a-Cloud.hdf5")
    model.load_weights("dwdii-emo-150-jhv5-9tf-Cloud.hdf5")

### Produce Predictions

In [53]:
predictOutput = model.predict(testX, batch_size=32, verbose=1)

263/263 [==============================] - 5s     


In [54]:
# How many predictions?
len(predictOutput)

263

In [55]:
# Show an example of the prediction output
print predictOutput[0]

[  2.88955162e-11   8.26386519e-18   9.99909699e-01   6.64880873e-10
   6.71791713e-05   7.65165112e-14   2.31399081e-05   1.72253852e-20]


In [56]:
np.array(predictOutput[0]).argmax()

2

In [57]:
numEmo = dwdii_transforms.numericEmotions()
numEmo[2]

'neutral'

In [58]:
fileList[0]

'y38.jpg'

## Persist Predictions

In this section we save the prediction results to a file in the prescribed Kaggle competition format.

In [59]:
# For each prediction output, build the row for the CSV and add to our list for saving.
outputData = []
outputData.append(["Image", "Emotion"])
for i in range(len(predictOutput)):
    arPred = np.array(predictOutput[i])
    predictionProb = arPred.max()
    predictionNdx = arPred.argmax()
    predictedEmo = numEmo[predictionNdx]
    fileName = fileList[i]
    outputData.append([fileName,predictedEmo])

In [60]:
print outputData[0:10]

[['Image', 'Emotion'], ['y38.jpg', 'neutral'], ['KA.AN2.40.jpg', 'surprise'], ['KajalAgarwal_22.jpg', 'surprise'], ['FaridaJalal_149.jpg', 'happiness'], ['NA.HA2.203.jpg', 'surprise'], ['KM.FE2.24.jpg', 'surprise'], ['JayaBhaduri_42.jpg', 'neutral'], ['FaridaJalal_315.jpg', 'happiness'], ['186a.jpg', 'surprise']]


In [61]:
with open('../output/dwdii_predictions.csv', 'w') as mycsvfile:
    dw = csv.writer(mycsvfile)
    for row in outputData:
        dw.writerow(row)

### Appendix: Sanity Check Code

In [ ]:
from scipy import misc
path = testDataPath + "/JayaBhaduri_42.jpg"
img = misc.imread(path)
img = misc.imresize(img, (150, 150))

misc.imsave("test1000.jpg", img)